# Example of using Agent node

In [8]:
import asyncio
import os
from dotenv import load_dotenv
from langchain_core.tools import tool
from langgraph.graph import END, START, StateGraph
from aic_flow.graph.state import State
from aic_flow.nodes.ai import Agent
from aic_flow.nodes.code import PythonCode
from aic_flow.nodes.telegram import MessageTelegram


load_dotenv()


model_settings = {
    "model": "gpt-4o-mini",
    "api_key": os.getenv("OPENAI_API_KEY"),
}

## Structured output with vanilla JSON dict

In [2]:
so_schema = """
json_dict = {
    "type": "object",
    "title": "Person",
    "description": "A person",
    "properties": {
        "name": {"type": "string"},
    },
    "required": ["name"],
}
"""

so_config = {
    "schema_type": "json_dict",
    "schema_str": so_schema,
}

agent_node = Agent(
    name="agent",
    model_settings=model_settings,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(  # noqa: F704, PLE1142
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

{'name': 'John Doe'}

## Structured output with OpenAI JSON dict

In [3]:
so_schema = """
oai_json_schema = {
    "name": "get_person",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {"name": {"type": "string"}},
        "additionalProperties": False,
        "required": ["name"],
    },
}
"""

so_config = {
    "schema_type": "json_dict",
    "schema_str": so_schema,
}

agent_node = Agent(
    name="agent",
    model_settings=model_settings,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(  # noqa: F704, PLE1142
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

{'name': 'John Doe'}

## Structured output with Pydantic Models

In [4]:
so_schema = """
class Person(BaseModel):
    \"\"\"A Person.\"\"\"

    name: str
"""

so_config = {
    "schema_type": "json_dict",
    "schema_str": so_schema,
}

agent_node = Agent(
    name="agent",
    model_settings=model_settings,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(  # noqa: F704, PLE1142
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

Person(name='John Doe')

## Structured output with Typed dict

In [5]:
so_schema = """
class Person(TypedDict):
    \"\"\"A Person.\"\"\"

    name: str
"""

so_config = {
    "schema_type": "typed_dict",
    "schema_str": so_schema,
}

agent_node = Agent(
    name="agent",
    model_settings=model_settings,
    structured_output=so_config,
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)
config = {"configurable": {"thread_id": "123"}}
result = await agent_node(  # noqa: F704, PLE1142
    {"messages": [{"role": "user", "content": "What's your name?"}]}, config
)

result["structured_response"]

{'name': 'John Doe'}

## Use a node as a tool

Main differences of a graph node and a function tool: A graph node only
receives `state: dict` as input, and returns a dict, while a function tool
can have arbitrary inputs.

In [6]:
telegram_node = MessageTelegram(
    name="MessageTelegram",
    token=os.getenv("TELEGRAM_TOKEN"),
)

agent_node = Agent(
    name="agent",
    model_settings=model_settings,
    tools=[telegram_node],
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)

config = {"configurable": {"thread_id": "123"}}
result = await agent_node(  # noqa: F704, PLE1142
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    f"Say hello to {os.getenv('TELEGRAM_CHAT_ID')} using "
                    "message_telegram tool"
                ),
            }
        ]
    },
    config,
)

result["messages"][-2]

ToolMessage(content='{"message_id": 532, "status": "sent"}', name='MessageTelegram', id='7079dc64-e766-40b3-86ba-0172c644dff0', tool_call_id='call_KqrJqEqle5TImLTQOPB4ob6c')

## Use sub-graph as a tool

### Define a sub-graph

In [11]:
python_code_node = PythonCode(
    name="PythonCode",
    code="return {'messages': [{'role': 'ai', 'content': 'Hello, ' + state['outputs']['initial'] + '.'}]}",  # noqa: E501
)

tool_graph = StateGraph(State)
tool_graph.add_node("python_code", python_code_node)
tool_graph.add_edge(START, "python_code")
tool_graph.add_edge("python_code", END)

python_code_graph = tool_graph.compile()

### Wrap the sub-graph as a tool

In [12]:
@tool(parse_docstring=True)
def greet(name: str) -> dict:
    """Greet the user.

    Args:
        name: The name of the user to greet.
    """
    result = asyncio.run(
        python_code_graph.ainvoke(
            {"messages": [], "outputs": {"initial": name}}, config={}
        )
    )
    return result["outputs"]["PythonCode"]

### Use the tool in an agent

In [13]:
agent_node = Agent(
    name="agent",
    model_settings=model_settings,
    tools=[greet],
    system_prompt="Your name is John Doe.",
    checkpointer="memory",
)

config = {"configurable": {"thread_id": "123"}}
result = await agent_node(  # noqa: F704, PLE1142
    {
        "messages": [
            {
                "role": "user",
                "content": ("Hello, my name is John Doe"),
            }
        ]
    },
    config,
)

result["messages"]

[HumanMessage(content='Hello, my name is John Doe', additional_kwargs={}, response_metadata={}, id='844ee22a-4465-43bd-afe6-11721c0d9465'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uPrNHTh6k1l4hYl5YBT6zSLR', 'function': {'arguments': '{"name":"John Doe"}', 'name': 'greet'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 63, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BaL9qyB5zBluFx3NIXW4UbhSokl48', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--123242f9-cbe9-4887-824b-1356d4783ff8-0', tool_calls=[{'name': 'greet', 'args': {'name': 'John Doe'}, 'id': 'call_uPrNHTh6k1l4hYl5YBT6zSL